In [1]:
import requests
import json
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import pandas as pd
import os
import zipfile

In [2]:
zip_filename = 'data/recipes_cleaned.zip'
file_inside_zip = 'recipes_cleaned.json'  # Replace with the actual file name inside the ZIP archive

# Open the ZIP file and read the file inside it
with zipfile.ZipFile(zip_filename, 'r') as zipf:
    with zipf.open(file_inside_zip) as file:
        # Read and decode the file contents (assuming it's UTF-8 encoded text)
        data = file.read().decode('utf-8')

In [8]:
data = json.loads(data)

In [10]:
data[0]

{'name': 'Potato-Crab Chowder',
 'description': 'Soup for the soul!',
 'ingredients': "['butter', 'onion', 'garlic', 'potatoes', 'flour', 'milk', 'black pepper', 'nutmeg', 'creamed corn', 'low sodium chicken broth', 'lump crabmeat', 'cayenne pepper', 'parsley']",
 'ingredients_raw_str': '["2   tablespoons    butter","1   medium    onion","2   tablespoons   minced garlic","3   cups    potatoes, skin on, 1 inch cubed ","3   tablespoons    flour","2   cups    milk","1/2  teaspoon    black pepper","1/4  teaspoon    nutmeg","1 (14   ounce) can   creamed corn","1 (14   ounce) can   low sodium chicken broth","1 (8   ounce) can   lump crabmeat","1   dash    cayenne pepper","1   pinch    parsley"]',
 'servings': 6,
 'steps': "['Saute onion&amp; garlic in melted butter in large sauce pan for 4 minutes.', 'Add potato and saute 1 minute.', 'Sprinkle in flour, cook for 1 minute stirring constantly.', 'Add milk, pepper, nutmeg, corn and broth.', 'Bring to a simmer over med heat stirring frequently.'

In [ ]:
recipe_descriptions = [description for description in recipes['description']]
print(recipe_descriptions[:10])

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, min_df=5)
recipe_tfidf = vectorizer.fit_transform(recipe_descriptions)

In [ ]:
k = 40  # Same as class demo
U, s, vt = svds(recipe_tfidf, k=k)
recipe_vectors = normalize(U, axis=1)